In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re


#initialize empty dataframe
results = pd.DataFrame()

#unique url identifiers for each race
link = [50896,50901,50906,50911, 50916, 50921, 50926]

In [128]:
#lites

#Dealing with names, especially camel case names made this a lot more difficult than I was anticipating
#but I persevered, and I'm glad I did. This is going to be fun to analyze.
html = np.empty(len(link))
df_list = [None] * len(link)

for i,nums in enumerate(link):
    url = 'http://www.promotocross.com/mx/event-results/{}/26'.format(nums)
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html, 'lxml')

#This is where things got incredibly tricky. Names seemed easy, but 
#names like R J and McElrath made a 3 hour ordeal be a 10 hour ordeal.
    elements = soup.find_all("span", class_="athlete-name mobile".split())

    name_gen = "\n".join("{}".format(el.get_text()) for el in elements)
    
#regex to make all camel case standardized lower
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name_gen)
    name_glob = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
#I got a big blob of new line seperated text, so I split to form a list
    name = name_glob.splitlines()
    del name[0::2]
    
    
    #elements = soup.find_all("span", class_="field-content".split())
    #name_rank = "\n".join("{} {}".format(el['class'], el.get_text()) for el in elements)
#Pulling in their race 1 and 2 results. I tried this a bunch of different ways, but I had an epihony that
#made this very simple. I had to split a #-# format, and wasn't sure. Regex can do that really easily, unbeknownst to me...
    elements = soup.find_all("div", class_="field-content".split())
    spread = "\n".join("{}".format(el.get_text()) for el in elements)
    spread = re.findall('\d{1,2}\-\d{1,2}', spread)
    position_list = []
    for nums in spread:
        place = nums.split('-')
        position_list.append(place)

        
#Adding the name of the race to differentiate the data
    #locale = soup.find_all('p')
    locale = ('Location {}'.format(i+1))

    
    #place_list = place * len(name)
#Splitting the races and making two lists
    race1 = []
    race2 = []
    for element1,element2 in position_list: 
        race1.append(element1)
        race2.append(element2)
#Making a data frame our of all of these
    df = pd.DataFrame({'Name':name, 'Race 1':race1, 'Race 2':race2}).reset_index()
    df['Overall'] = df['index'] + 1
    df.set_index('Name')
    df.columns = [str(col) + ' ' + str(locale) for col in df.columns]
    df_list[i] = df
        


for i,dfs in enumerate(df_list):
#setting up a join on name to help us predict expected finish for various riders
    if i == 0:
        df_joined = dfs
               
    else:
        df_joined = df_joined.merge(dfs, left_index=True, right_index = True,how = 'outer')
#setting up a concatenated dataframe for predicting single race outcome 
    
    if i == 0:
        dfs.columns = ['Index', 'Name','Race 1', 'Race 2', 'Overall']
        df = dfs
    else:
        dfs.columns = ['Index', 'Name','Race 1', 'Race 2', 'Overall']
        df = pd.concat([df,dfs], axis=0)
    
    


In [129]:
df_joined = df_joined.drop(['Name Location 2','Name Location 3','Name Location 4','Name Location 5','Name Location 6', 'Name Location 7'],1)
df_joined = df_joined.drop(['Index','index Location 2','index Location 3','index Location 4','index Location 5','index Location 6', 'index Location 7'],1)


In [130]:
df_joined

,Name,Race 1,Race 2,Overall,Race 1 Location 2,Race 2 Location 2,Overall Location 2,Race 1 Location 3,Race 2 Location 3,Overall Location 3,...,Overall Location 4,Race 1 Location 5,Race 2 Location 5,Overall Location 5,Race 1 Location 6,Race 2 Location 6,Overall Location 6,Race 1 Location 7,Race 2 Location 7,Overall Location 7
0,zach _osborne,1,1,1,4,2,1.0,4,1,1.0,...,1.0,1,2,1.0,2,1,1.0,2,1,1.0
1,alex _martin,4,4,2,1,7,2.0,1,4,2.0,...,2.0,5,1,2.0,3,3,2.0,1,2,2.0
2,aaron _plessinger,3,5,3,9,1,3.0,3,2,3.0,...,3.0,3,3,3.0,1,8,3.0,3,3,3.0
3,adam _cianciarulo,2,8,4,7,3,4.0,2,3,4.0,...,4.0,2,5,4.0,7,2,4.0,5,4,4.0
4,austin _forkner,11,2,5,5,4,5.0,8,5,5.0,...,5.0,4,4,5.0,5,4,5.0,4,6,5.0
5,colt _nichols,5,6,6,6,5,6.0,9,6,6.0,...,6.0,8,8,6.0,6,11,6.0,7,5,6.0
6,joey _savatgy,7,7,7,3,13,7.0,6,12,7.0,...,7.0,6,11,7.0,4,14,7.0,6,7,7.0
7,justin _hill,8,9,8,8,10,8.0,11,8,8.0,...,8.0,11,7,8.0,9,9,8.0,9,8,8.0
8,mitchell _oldenburg,6,12,9,2,29,9.0,10,9,9.0,...,9.0,13,6,9.0,12,7,9.0,8,10,9.0
9,mitchell _harrison,9,11,10,13,8,10.0,12,10,10.0,...,10.0,10,10,10.0,11,10,10.0,15,9,10.0


In [131]:
df.columns

Index([u'Index', u'Name', u'Race 1', u'Race 2', u'Overall'], dtype='object')

In [132]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_curve, classification_report
from yellowbrick import ROCAUC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import SelectKBest
from yellowbrick.classifier import ClassificationReport
from sklearn.ensemble import RandomForestRegressor

X = pd.to_numeric(df['Race 1']).values.reshape(-1,1)
y = df['Race 2'].astype(int)




clf = LinearRegression()

#pointless pipeline. Still grasping their usage, so I just wanted to make sure I understood
#the basics. Will try and implement a real one in the next step.
steps = [('LinearRegression', clf)]

pipeline = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



### fit your pipeline on X_train and y_train
pipeline.fit( X_train, y_train )
### call pipeline.predict() on X_test data to make a set of test predictions
y_prediction = pipeline.predict( X_test )
y_prediction = y_prediction.astype(int)
print(y_prediction)
### test your predictions using classification_report()
score = accuracy_score( y_test, y_prediction )
### and print the report
print(score)




[35 30 11 21 20 12 25 10 23 32 14 16 22 18  6 32 22 14 34 22 20 26 21 26 10
 30 25 29 17 29  9 33 18 19 10 27 20 22 30 16 32 16 34 32 26 12 13 10  6 24
  7 16 25 29 18 30 12 31 25  9 10 34 13  7 16 31 33 33 16 33 16 26 14 26 24
 32 29 30 18 30 11  6  6 18 17]
0.105882352941


In [133]:

X = pd.to_numeric(df['Race 1']).values.reshape(-1,1)
y = df['Race 2'].astype(int)




clf = RandomForestRegressor()

#pointless pipeline. Still grasping their usage, so I just wanted to make sure I understood
#the basics. Will try and implement a real one in the next step.
steps = [('Forest', clf)]

pipeline = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



### fit your pipeline on X_train and y_train
pipeline.fit( X_train, y_train )
### call pipeline.predict() on X_test data to make a set of test predictions
y_prediction = pipeline.predict( X_test )
y_prediction = y_prediction.astype(int)
print(y_prediction)
### test your predictions using classification_report()
score = accuracy_score( y_test, y_prediction )
### and print the report
print(score)


[20 26  9 25 17  8 26 20 24 36 12 12 22 17  1 36 22 12 20 22 17 25 25 25 10
 26 26 32 18 31 13 23 15 23 20 28 17 22 26 17 36 12 20 33 30  8 11 20  1 25
  8 17 26 32 15 26  8 34 26 13 10 29 11  8 12 34 23 23 17 23 12 30 12 25 29
 33 32 26 17 26  9  1  1 15 18]
0.0588235294118


Unsurprisingly, the linear regression is a pretty terrible predictor. I don't think the model is bad as much as I know the actual outcomes of a given race are so uncertain that it's foolish to try and predict them. Using a forest regressor, I got even worse results, so it's still futile. 

What I will do, however, is split the groups into 1-9,10-19,20-29,30-39, and 40. This, I think will yield really good results. I can also use a a classification algorithm, such as SVM or RandomForest, which are notably better at these kind of things.

In [134]:
df['Race 2'] = pd.to_numeric(df['Race 2'])
df['position'] = df['Race 2']//10



In [135]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

steps = [('SVM', SVC())]

pipeline = Pipeline(steps)
params = {'SVM__C':[1,10,100],
            'SVM__gamma':[.1,.01]}
                                                                      

X = df['Race 1'].values.reshape(-1,1)
y = df['position']


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .3)                                                                      
                                                                      
svm_cv = GridSearchCV(pipeline, params)
svm_cv.fit(x_train, y_train)

y_pred = svm_cv.predict(X_test)

print(accuracy_score(y_test,y_pred))
                                                                      
print(y_test.values,y_pred)                                                                      
                                                                      
                                                                      

0.294117647059
(array([0, 3, 3, 1, 2, 3, 2, 2, 2, 2, 3, 1, 1, 1, 0, 1, 3, 0, 3, 2, 4, 1, 3,
       1, 1, 0, 0, 3, 2, 0, 3, 0, 2, 3, 4, 1, 2, 1, 1, 2, 1, 1, 1, 3, 3, 4,
       1, 1, 3, 2, 0, 1, 3, 3, 0, 3, 2, 2, 0, 0, 3, 3, 1, 3, 3, 1, 3, 2, 2,
       3, 1, 2, 1, 3, 1, 1, 2, 0, 1, 2, 1, 0, 3, 3, 1]), array([2, 3, 0, 2, 1, 0, 2, 0, 2, 3, 1, 1, 2, 1, 0, 3, 2, 1, 2, 2, 1, 2, 2,
       2, 0, 3, 2, 3, 1, 3, 0, 3, 1, 1, 0, 2, 1, 2, 3, 1, 3, 1, 2, 3, 2, 0,
       0, 0, 0, 2, 0, 1, 2, 3, 1, 3, 0, 3, 2, 0, 0, 3, 0, 0, 1, 3, 3, 3, 1,
       3, 1, 2, 1, 2, 2, 3, 3, 3, 1, 3, 0, 0, 0, 1, 1]))


SVM didn't do super well, but what about a forest?

In [136]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

steps = [('forest', RandomForestClassifier())]

pipeline = Pipeline(steps)
params = {'forest__n_estimators':[1,5,10],
            'forest__max_features':['auto','sqrt']}
                                                                      

X = df['Race 1'].values.reshape(-1,1)
y = df['position']


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .3)                                                                      
                                                                      
rf_cv = GridSearchCV(pipeline, params)
rf_cv.fit(x_train, y_train)

y_pred = rf_cv.predict(X_test)

print(accuracy_score(y_test,y_pred))
                                                                      
print(y_test.values,y_pred)                                                                      
                                                                      
                            

0.235294117647
(array([3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 4, 0, 0, 3, 3, 0, 3, 0, 2, 2, 2, 1, 2,
       0, 2, 2, 2, 1, 3, 3, 0, 0, 1, 2, 0, 2, 1, 0, 3, 1, 2, 3, 0, 2, 0, 2,
       1, 0, 0, 1, 1, 2, 3, 2, 2, 3, 0, 3, 1, 0, 1, 0, 2, 3, 0, 0, 0, 3, 1,
       1, 2, 4, 3, 2, 4, 2, 2, 2, 2, 0, 1, 2, 4, 2, 0]), array([2, 3, 0, 2, 3, 0, 2, 0, 2, 3, 1, 1, 2, 1, 0, 3, 2, 1, 1, 2, 3, 2, 2,
       2, 1, 3, 2, 3, 1, 3, 0, 2, 1, 1, 0, 2, 3, 2, 3, 1, 3, 1, 1, 3, 2, 0,
       1, 0, 0, 2, 0, 1, 2, 3, 1, 3, 0, 3, 2, 0, 1, 3, 1, 0, 1, 3, 2, 2, 1,
       2, 1, 2, 1, 2, 3, 3, 3, 3, 1, 3, 0, 0, 0, 1, 1]))


So it looks like SVM is our winner. I'm not going to lie, I'm still figuring out the best way to tune, so I almost certainly missed something, but given what I know at this exact moment in time, I believe SVM is doing well (at least from an accuracy score perspective).

Now let's move onto our other data frame. Here, I think our best move will be to make a regressor that takes in the various standings for each rider and predicts their seasonal average. 

In [137]:
df_joined.head()

,Name,Race 1,Race 2,Overall,Race 1 Location 2,Race 2 Location 2,Overall Location 2,Race 1 Location 3,Race 2 Location 3,Overall Location 3,...,Overall Location 4,Race 1 Location 5,Race 2 Location 5,Overall Location 5,Race 1 Location 6,Race 2 Location 6,Overall Location 6,Race 1 Location 7,Race 2 Location 7,Overall Location 7
0,zach _osborne,1,1,1,4,2,1.0,4,1,1.0,...,1.0,1,2,1.0,2,1,1.0,2,1,1.0
1,alex _martin,4,4,2,1,7,2.0,1,4,2.0,...,2.0,5,1,2.0,3,3,2.0,1,2,2.0
2,aaron _plessinger,3,5,3,9,1,3.0,3,2,3.0,...,3.0,3,3,3.0,1,8,3.0,3,3,3.0
3,adam _cianciarulo,2,8,4,7,3,4.0,2,3,4.0,...,4.0,2,5,4.0,7,2,4.0,5,4,4.0
4,austin _forkner,11,2,5,5,4,5.0,8,5,5.0,...,5.0,4,4,5.0,5,4,5.0,4,6,5.0


In [155]:
df_race_res = df_joined.drop(['Overall', 'Overall Location 2','Overall Location 3','Overall Location 4','Overall Location 5','Overall Location 6','Overall Location 7'],1)

In [156]:

df_race_res.iloc[:,1:] = df_race_res.iloc[:,1:].applymap(pd.to_numeric)
df_race_res['avg_finish'] = df_race_res.iloc[:,1:].mean(axis = 1)

In [157]:
type(df_race_res.iloc[1,1])

float

In [158]:
df_race_res

,Name,Race 1,Race 2,Race 1 Location 2,Race 2 Location 2,Race 1 Location 3,Race 2 Location 3,Race 1 Location 4,Race 2 Location 4,Race 1 Location 5,Race 2 Location 5,Race 1 Location 6,Race 2 Location 6,Race 1 Location 7,Race 2 Location 7,avg_finish
0,zach _osborne,1,1,4,2,4,1,2,1,1,2,2,1,2,1,1.785714
1,alex _martin,4,4,1,7,1,4,1,2,5,1,3,3,1,2,2.785714
2,aaron _plessinger,3,5,9,1,3,2,3,3,3,3,1,8,3,3,3.571429
3,adam _cianciarulo,2,8,7,3,2,3,4,4,2,5,7,2,5,4,4.142857
4,austin _forkner,11,2,5,4,8,5,7,5,4,4,5,4,4,6,5.285714
5,colt _nichols,5,6,6,5,9,6,6,6,8,8,6,11,7,5,6.714286
6,joey _savatgy,7,7,3,13,6,12,8,7,6,11,4,14,6,7,7.928571
7,justin _hill,8,9,8,10,11,8,9,8,11,7,9,9,9,8,8.857143
8,mitchell _oldenburg,6,12,2,29,10,9,5,13,13,6,12,7,8,10,10.142857
9,mitchell _harrison,9,11,13,8,12,10,10,9,10,10,11,10,15,9,10.500000


In [161]:
X = df_race_res.iloc[:,1:-1]
y = df_race_res.iloc[:,-1]

In [167]:

forest = RandomForestRegressor()

params = {'forest__n_estimators':[1,5,10],
            'forest__max_features':['auto','sqrt']}


steps = [('forest', clf)]

pipeline = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

                                                                      
rf_cv = GridSearchCV(pipeline, params)
rf_cv.fit(X_train, y_train)

y_prediction = rf_cv.predict( X_test )


score = accuracy_score( y_test, y_prediction )
print(score)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').